# Creating the HTLC-Timeout Transaction

In this section, we'll build a Lightning channel HTLC-timeout transaction from scratch using Python. We'll walk through each part of the transaction — how it's constructed and signed. The process will be tested using Bitcoin Core in regtest mode.

## Setup

For this notebook, we'll use the commitment transaction created in `chapter 3 - in-flight htlc commitment transaction` which contains the offered HTLC output that we'll spend with the HTLC-timeout transaction.

In [1]:
%run "../chapter 3 - in-flight htlc commitment transaction/in-flight htlc commitment transaction.ipynb"

2026-02-15T15:44:17.923000Z TestFramework (INFO): PRNG seed is: 8736749881137389748
2026-02-15T15:44:17.926000Z TestFramework (INFO): Initializing test directory /tmp/bitcoin_func_test_ult9ry11
🟢 New TestShell started. Block height: 0
Alice Per Commitment Seed 34b581ec20bf2c6cae3d4d4dcbfddc8a3727a1e9a57c55f3520e770607898c06
Bob Per Commitment Seed 89c994b3ddad4698acee71e42d8bcace48eea739caaba371eb110e77663ec56d
Alice Payment BasePoint:  025f892a06124391e2f38ce35d943cdc09f63e203330dbd9cb6113a903e0738458
Bob Payment BasePoint:  02f98efd3f2b2fbe7bd83c419f5f64f8280798b8a9175fdb77c0091bbb95c79506
To obscure commitment number 0xb433fd43a66f
Alice funding pubkey: f3621ef22dc50be90b4c969b77731cedb8d66fee72663f876daf6a561dd000a2
Alice funding privkey: 2438d45c303d60bdbd08bc711cb079e60781d51a0709b970b81a2555a6657ef7
Alice funding address: bcrt1p7d3pau3dc597jz6vj6dhwucuakudvmlwwfnrlpmd4a49v8wsqz3qf9ddfs
Alice sweeper pubkey: f86a51744713efe7c188cd6e0177122adb901c8990570aac650cc060801c14c5
Alice s

aggregated Schnorr verifies? True
signed tx:  02000000000101a4e54c7591bce004c947c40a90a5263efeb395ec38b4760dcdb5406dc575df350000000000fd33b480024a0100000000000022512063b192011ae91765a071d5ea356e7d5d6e04c28c8632a44b74bcf866230c62f65a400f0000000000225120cb56fad21e2ac194fe207224aad544008a3774437465a7f229adea4df883dc180140e6223d9ae1b7b039fbb9230ca3b792f5ca00769b46fbbeb24027bb5c1e657af661318847d9f13d96f8eb99a752ac92dc2ca0f999a07bb2a1c412646e15bd758e6fa64320
{
  "txid": "985e164264c4fa59f352408780ee577db6123d633bf8c96997a95380df0832f9",
  "hash": "4dbaf0da63f39271386cfdc135dfdd0085748b74927760e311e83a04ac10f7b5",
  "version": 2,
  "size": 205,
  "vsize": 154,
  "weight": 616,
  "locktime": 541304431,
  "vin": [
    {
      "txid": "35df75c56d40b5cd0d76b438ec95b3fe3e26a5900ac447c904e0bc91754ce5a4",
      "vout": 0,
      "scriptSig": {
        "asm": "",
        "hex": ""
      },
      "txinwitness": [
        "e6223d9ae1b7b039fbb9230ca3b792f5ca00769b46fbbeb24027bb5c1e657af661318847d9f13d96f

## HTLC-Timeout Transaction

The HTLC-timeout transaction is a second-level transaction that spends an offered HTLC output from a commitment transaction. It allows the local party (Alice, who offered the HTLC) to reclaim the funds after the timeout period if the remote party (Bob) doesn't claim the HTLC with the preimage.

### The Unsigned HTLC-Timeout Transaction

#### The Input

The input is the offered HTLC output from Alice's commitment transaction (output index 3).

In [2]:
# Get the commitment transaction details
# Alice's second commitment transaction has the offered HTLC output at index 3
commitment_txid = decoded_alice_commitment_tx['txid']
commitment_output_index = 3

print(f"Spending from commitment tx: {commitment_txid}")

# VERSION
# version '2' indicates that we may use relative timelocks (BIP68)
version = bytes.fromhex("0200 0000")

# MARKER (new to segwit)
marker = bytes.fromhex("00")

# FLAG (new to segwit)
flag = bytes.fromhex("01")

# INPUTS
# We have just 1 input
input_count = bytes.fromhex("01")

# Convert txid and index to bytes (little endian)
commitment_txid = bytes.fromhex(commitment_txid)[::-1]
commitment_index = commitment_output_index.to_bytes(4, byteorder="little", signed=False)

# For the unsigned transaction we use an empty scriptSig
scriptsig = bytes.fromhex("")

# Sequence: must be set to enforce relative locktime (though scriptA doesn't have CSV)
# Set to 1 for option_anchors
sequence_htlc = bytes.fromhex("01000000")

inputs = (
    commitment_txid
    + commitment_index
    + varint_len(scriptsig)
    + scriptsig
    + sequence_htlc
)

Spending from commitment tx: 6c5c50a48d8ea27f09e69c08da5e44da810de8112ea64cce9756fa27c2bdf16b


#### The Output

The HTLC-timeout transaction has a single output that pays to Alice with a delay. This output is similar to the `to_local` output in commitment transactions: it's spendable by Alice after a delay, or immediately by Bob if he has the revocation key (in case this commitment gets revoked).

#### To Local Delayed Output

    +------+---------------+
    | OP_1 |       Q       |
    +------+---------------+
                   ^
                   |   +-------------------+
                    ---| P(revocation) + T |
                       +-------------------+
                                         ^
                                         |
                                   +-----------+        
                                   | T = t * G |
                                   +-----------+        
                                         ^
                                         |
     +---+   +-------------------------------------------------------+
     | t | = | TaggedHash ("Taptweak", P(revocation) || script_root) |
     +---+   +-------------------------------------------------------+
                                                             ^
                                                             |
                                                          +-----+
                                                          |  h  |
                                                          +-----+                   
                                                             ^                                                          
                                                             |                                                          
      +-----------------------------------------------------------+                                           
      | P(local_delayed) OP_CHECKSIG to_self_delay OP_CSV OP_DROP |                                           
      +-----------------------------------------------------------+

Obs: If option_anchors applies, which is the case here, then the HTLC-timeout and HTLC-success transactions are signed with the input and output having the same value. This means they have a zero fee and MUST be combined with other inputs to arrive at a reasonable fee.

In [ ]:
# Output count
output_count = bytes.fromhex("01")

# Output value
output_value_sat = htlc_output_value_satoshis
output_value = output_value_sat.to_bytes(8, byteorder="little", signed=False)

# We already have alice_delayed_pubkey and bob_revocation_pubkey from chapter 3
# These are the same keys used in the commitment transaction to_local output

# Create the script: P(local_delayed) OP_CHECKSIG to_self_delay OP_CSV OP_DROP
script_output = CScript([alice_delayed_pubkey.get_bytes(bip340=True), OP_CHECKSIG, to_self_delay, OP_CHECKSEQUENCEVERIFY, OP_DROP])

# Compute output script_root
hash_input = TAPSCRIPT_VER + ser_string(script_output)
script_root = tagged_hash("TapLeaf", hash_input)

# Compute the output Tagged Hash
taptweak = tagged_hash("TapTweak", bob_revocation_pubkey.get_bytes(bip340=True) + script_root)
htlc_bob_revocation_pubkey_tweaked = bob_revocation_pubkey.tweak_add(taptweak)
# Compute scriptPubKey P2TR: OP_1 (0x51) + PUSH32 (0x20) + xonly(32B)
output_spk = bytes.fromhex("51") + varint_len(htlc_bob_revocation_pubkey_tweaked.get_bytes(bip340=True)) + htlc_bob_revocation_pubkey_tweaked.get_bytes(bip340=True)

outputs = (
    output_value
    + varint_len(output_spk)
    + output_spk
)

# Locktime: set to current block height + time lock delta
current_block_height = node.getblockcount()
locktime = current_block_height + 60  # assuming a 60-block of time lock delta
locktime = locktime.to_bytes(4, 'little')

unsigned_tx = (
    version
    + input_count
    + inputs
    + output_count
    + outputs
    + locktime
)
print("\nunsigned_tx:", unsigned_tx.hex())

# Decode the unsigned transaction to verify it looks correct
decoded_htlc_timeout = node.decoderawtransaction(unsigned_tx.hex())
print(json.dumps(decoded_htlc_timeout, indent=2, default=str))


unsigned_tx: 02000000016bf1bdc227fa5697ce4ca62e11e80d81da445eda089ce6097fa28e8da4505c6c0300000000010000000120a10700000000002251209c4e1af0245ce5eace1cc1432fe0d0140ca6de1e289aa4e5d34c19d4b7efafdfa3000000
{
  "txid": "cb66bccaf8a0fa84adbaf42edc0c6dc4c9e94b1ebb61b74fd88fb81eb0f9b059",
  "hash": "cb66bccaf8a0fa84adbaf42edc0c6dc4c9e94b1ebb61b74fd88fb81eb0f9b059",
  "version": 2,
  "size": 94,
  "vsize": 94,
  "weight": 376,
  "locktime": 163,
  "vin": [
    {
      "txid": "6c5c50a48d8ea27f09e69c08da5e44da810de8112ea64cce9756fa27c2bdf16b",
      "vout": 3,
      "scriptSig": {
        "asm": "",
        "hex": ""
      },
      "sequence": 1
    }
  ],
  "vout": [
    {
      "value": "0.00500000",
      "n": 0,
      "scriptPubKey": {
        "asm": "1 9c4e1af0245ce5eace1cc1432fe0d0140ca6de1e289aa4e5d34c19d4b7efafdf",
        "desc": "rawtr(9c4e1af0245ce5eace1cc1432fe0d0140ca6de1e289aa4e5d34c19d4b7efafdf)#c0d4ugzn",
        "hex": "51209c4e1af0245ce5eace1cc1432fe0d0140ca6de1e289aa4e5d34c19

## The sighash for script path spend

Unlike the commitment transaction which uses **key path spending**, the HTLC-timeout transaction uses **script path spending** through the Taproot tree. We need to spend using scriptA of the offered HTLC output.

The sighash calculation for script path spending is similar to key path, but includes additional data about the script being executed.

In [4]:
# We're spending the offered HTLC output using scriptA
# Recall scriptA from chapter 3: P(local_htlc) OP_CHECKSIGVERIFY P(remote_htlc) OP_CHECKSIG
spending_script = CScript([alice_htlc_pubkey.get_bytes(bip340=True), OP_CHECKSIGVERIFY, bob_htlc_pubkey.get_bytes(bip340=True), OP_CHECKSIG])

# Calculate the tapleaf hash for the script we're spending
hash_input_spending = TAPSCRIPT_VER + ser_string(spending_script)
tapleaf_hash = tagged_hash("TapLeaf", hash_input_spending)
key_version = bytes.fromhex("00")  # reserved for future ugrades
codeseparator = bytes.fromhex("ffffffff")  

# SIGHASH for script path spend (BIP-341)
index_of_this_input = bytes.fromhex("0000 0000")
sighash_epoch = bytes.fromhex("00")
hash_type = bytes.fromhex("83")  # SIGHASH_SINGLE|SIGHASH_ANYONECANPAY (0x03 | 0x80)

# When SIGHASH_SINGLE (0x03) is set, these fields are 35 bytes (BIP-341)
# corresponding to the output (32): the SHA256 of the corresponding output in CTxOut format.
sha_outputs = sha256(outputs).digest()

# Data about this input, script path (no annex)
spend_type = bytes.fromhex("02")

# Common signature message extension (BIP-341)
# When SIGHASH_ANYONECANPAY is set, we include data about THIS specific input:
outpoint = commitment_txid + commitment_index  # 36 bytes
amount = htlc_output_value  # 8 bytes
nSequence = sequence_htlc  # 4 bytes

sig_msg = (
    sighash_epoch
    + hash_type
    + version
    + locktime
    + spend_type
    + outpoint          
    + amount
    + varint_len(alice_offered_htlc_spk)
    + alice_offered_htlc_spk
    + nSequence
    + sha_outputs
    + tapleaf_hash
    + key_version
    + codeseparator
)

print("\nSignature message:", sig_msg.hex())

tag_hash = sha256("TapSighash".encode()).digest()
sighash = sha256(tag_hash + tag_hash + sig_msg).digest()
print("\nSighash:", sighash.hex())


Signature message: 008302000000a3000000026bf1bdc227fa5697ce4ca62e11e80d81da445eda089ce6097fa28e8da4505c6c0300000020a1070000000000225120ba9dfd92bdd3bb7b3b4faa510e1076c32c5105c0f0e620d238486e8fa84673830100000037b1a6d47e215cb28259eac50b4162048f0c004a4b0fde33a8532cd25b5a1f1b509fbe809f2233d8daa3bca96c2735a7f7a1bd75eb48976978029ac95cd0fd1300ffffffff

Sighash: be20b7af110c1ab9790870be2e074dc252f2e9edcf8ef5f01dfb630ebc9626c9


## Signing the sighash

For the HTLC-timeout transaction, we need signatures from both Alice's and Bob's HTLC keys. These are regular Schnorr signatures (not MuSig2 aggregated signatures), since the script requires two separate signature checks.

In [5]:
# Derive Alice's HTLC private key
alice_htlc_basepoint = derivate_key(alice_node_seed, family=2, channel_index=0)
alice_htlc_privkey = alice_htlc_basepoint.get_privkey(alice_per_commitment.get_pub())

# Derive Bob's HTLC private key
bob_htlc_basepoint = derivate_key(bob_node_seed, family=2, channel_index=0)
bob_htlc_privkey = bob_htlc_basepoint.get_privkey(bob_per_commitment.get_pub())

# Generate auxiliary random data for each signature
import secrets
aux_alice = secrets.token_bytes(32)
aux_bob = secrets.token_bytes(32)

# Sign with Alice's HTLC key
alice_htlc_sig = alice_htlc_privkey.sign_schnorr(sighash, aux_alice)
# APPEND sighash type byte for non-default sighash!
alice_htlc_sig = alice_htlc_sig + hash_type  # Add 0x83 byte

# Verify Alice's signature (without the sighash byte for verification)
alice_sig_valid = alice_htlc_pubkey.verify_schnorr(alice_htlc_sig[:-1], sighash)
print("Alice signature valid?", alice_sig_valid)
print(f"Alice signature length: {len(alice_htlc_sig)} bytes (should be 65)")

# Sign with Bob's HTLC key  
bob_htlc_sig = bob_htlc_privkey.sign_schnorr(sighash, aux_bob)
# APPEND sighash type byte for non-default sighash!
bob_htlc_sig = bob_htlc_sig + hash_type  # Add 0x83 byte

# Verify Bob's signature (without the sighash byte for verification)
bob_sig_valid = bob_htlc_pubkey.verify_schnorr(bob_htlc_sig[:-1], sighash)
print("Bob signature valid?", bob_sig_valid)
print(f"Bob signature length: {len(bob_htlc_sig)} bytes (should be 65)")

Alice signature valid? True
Alice signature length: 65 bytes (should be 65)
Bob signature valid? True
Bob signature length: 65 bytes (should be 65)


## The signed transaction

Now we construct the witness for the script path spend. The witness stack for spending scriptA contains:
1. Bob's signature (last signature on stack, for OP_CHECKSIG)
2. Alice's signature (for OP_CHECKSIGVERIFY)
3. The script being executed (scriptA)
4. The control block (proving scriptA is in the taproot tree)

In [6]:
# Construct the control block
# Control block format: <version byte> <internal key> <parity bit> [<merkle proof>]
# Version byte: 0xc0 | (parity of Q)

# Get parity bit from Q
parity = 0 if bob_revocation_pubkey_tweaked.get_bytes(bip340=False)[0] == 0x02 else 1
version_byte = bytes([0xc0 | parity])

control_block = version_byte + bob_revocation_pubkey.get_bytes(bip340=True) + htlc_taggedhash_leafB

print("\ncontrol_block:", control_block.hex())
print("\nspending_script:", spending_script.hex())

# Construct witness stack
# Stack order (bottom to top): <alice_sig> <bob_sig> <script> <control_block>
witness = (
    bytes.fromhex("04")  # 4 stack items
    + varint_len(bob_htlc_sig)    # Bob's signature (for OP_CHECKSIG)
    + bob_htlc_sig
    + varint_len(alice_htlc_sig)  # Alice's signature (for OP_CHECKSIGVERIFY)
    + alice_htlc_sig
    + varint_len(spending_script)  # The script
    + spending_script
    + varint_len(control_block)    # The control block
    + control_block
)

# The final signed transaction
signed_tx = (
    version
    + marker
    + flag
    + input_count
    + inputs
    + output_count
    + outputs
    + witness
    + locktime
)

print("\nsigned tx:", signed_tx.hex())

# Before testing it, we need to confirm the commitment transaction so that the HTLC output exists on-chain
alice_commitment_txid = node.sendrawtransaction(signed_alice_commitment_tx.hex())
result = node.generatetoaddress(nblocks=1, address=address, called_by_framework=True)

print("alice_commitment_txid: ", alice_commitment_txid)

# Now we can test if the HTLC timeout transaction would be accepted in the mempool (it shouldn't, since the commitment tx has not enought confirmations yet)
print("\nTest mempool accept:")
try:
    result = node.testmempoolaccept(rawtxs=[signed_tx.hex()])
    print(json.dumps(result, indent=2, default=str))
except Exception as e:
    print(f"Expected error (commitment tx not in mempool): {e}")



control_block: c1c39a6b0cffa569f243cec8252a1e5f93b4b072247ff18773917b72fb6341f6088da58f2aee0bccdd60c6065603a5d2ab655e79971b068228afbccbb02220f54b

spending_script: 207546c6ab3a8c005ce24e9feb7154fd35b62ac0f9125beb5e692d4d912a95de77ad207bd8383df06640c0efe760bb2bac1a1585bdc664e6f1cb61c41a76a6f2be3484ac

signed tx: 020000000001016bf1bdc227fa5697ce4ca62e11e80d81da445eda089ce6097fa28e8da4505c6c0300000000010000000120a10700000000002251209c4e1af0245ce5eace1cc1432fe0d0140ca6de1e289aa4e5d34c19d4b7efafdf044146c08745896cf49dd44df569af2dcf0db716fa3451c5e78907c63a82048dbf3ab535b8b0001550be40b43a5191ed079d2fe666bf8486aaf6e61a3c477500693783418b3c08268c5da6cdc603c678cee2bf3fafab9c28c71b811f9f98a3771aa035ae6877b146ba7661cec40a49c00e338f2ad68cb140262bb3606d010eb19496811e8344207546c6ab3a8c005ce24e9feb7154fd35b62ac0f9125beb5e692d4d912a95de77ad207bd8383df06640c0efe760bb2bac1a1585bdc664e6f1cb61c41a76a6f2be3484ac41c1c39a6b0cffa569f243cec8252a1e5f93b4b072247ff18773917b72fb6341f6088da58f2aee0bccdd60c6065603a5d2

In [7]:
# We mine enough blocks to satisfy the htlc time lock delta but now we don´t fit the min relay fee.
# We need actually to add another input to pay the fee.
result = node.generatetoaddress(nblocks=59, address=address, called_by_framework=True)

print("\nTest mempool accept:")
try:
    result = node.testmempoolaccept(rawtxs=[signed_tx.hex()])
    print(json.dumps(result, indent=2, default=str))
except Exception as e:
    print(f"Expected error (commitment tx not in mempool): {e}")


Test mempool accept:
[
  {
    "txid": "cb66bccaf8a0fa84adbaf42edc0c6dc4c9e94b1ebb61b74fd88fb81eb0f9b059",
    "wtxid": "42b8f51bcb5439a1bcb38a162ee10c6693ac628676e5fae469da0ddb6ce85c14",
    "allowed": false,
    "reject-reason": "min relay fee not met",
    "reject-details": "min relay fee not met, 0 < 162"
  }
]


## Adding input for fees

As stated in the BOLTs: "If option_anchors applies, then the HTLC-timeout and HTLC-success transactions are signed with the input and output having the same value. This means they have a zero fee and MUST be combined with other inputs to arrive at a reasonable fee."

We'll now change the transaction adding alice sweeper output created on chapter 1 as a second input to pay for fees.

In [8]:
# INPUTS
# We have 2 inputs: the HTLC output and alice sweeper output 
input_count = bytes.fromhex("02")


# INPUT 2: alice sweeper output
# Get funding transaction details from chapter 1
alice_txid_to_spend_bytes = bytes.fromhex(txid_to_spend)[::-1]
sweeper_index_bytes = (alice_sweeper_index).to_bytes(4, byteorder="little", signed=False)
sequence_change = bytes.fromhex("ffffffff")

inputs_v2 = (
    commitment_txid
    + commitment_index
    + varint_len(scriptsig)
    + scriptsig
    + sequence_htlc
    + alice_txid_to_spend_bytes
    + sweeper_index_bytes
    + varint_len(scriptsig)
    + scriptsig
    + sequence_change
)

print(f"Added alice_change input from funding tx: {funding_channel_txid}:1")

Added alice_change input from funding tx: 35df75c56d40b5cd0d76b438ec95b3fe3e26a5900ac447c904e0bc91754ce5a4:1


In [9]:
# OUTPUTS
# Now we have 2 outputs: HTLC delayed output + change back to alice
output_count_v2 = bytes.fromhex("02")

# OUTPUT 1: HTLC delayed output (same as before)
# Reuse the same output_value and output_spk from v1

# OUTPUT 2: Change back to alice_change address
# Calculate: alice_change_value (98,999,700) + htlc_value (500,000) - htlc_output (500,000) - fee
tx_fee_sat = 300
alice_change_value = int(sweeper_initial_fund * 100000000) - tx_fee_sat
alice_change_value = alice_change_value.to_bytes(8, byteorder="little", signed=False)

# scriptPubKey P2TR: OP_1 (0x51) + PUSH32 (0x20) + alice_change_pubkey
output_spk_change = bytes.fromhex("51") + varint_len(alice_change_pubkey.get_bytes(bip340=True)) + alice_change_pubkey.get_bytes(bip340=True)

# Use outputs_v2 to avoid overwriting the original outputs variable
outputs_v2 = (
    output_value
    + varint_len(output_spk)
    + output_spk
    + alice_change_value
    + varint_len(output_spk_change)
    + output_spk_change
)

print(f"Transaction fee: {tx_fee_sat} sats")

Transaction fee: 300 sats


In [10]:
unsigned_tx = (
    version
    + input_count
    + inputs_v2
    + output_count_v2
    + outputs_v2
    + locktime
)

print("unsigned_tx_v2:", unsigned_tx.hex())

# Decode the unsigned transaction to verify it looks correct
decoded_htlc_timeout = node.decoderawtransaction(unsigned_tx.hex())
print(json.dumps(decoded_htlc_timeout, indent=2, default=str))

unsigned_tx_v2: 02000000026bf1bdc227fa5697ce4ca62e11e80d81da445eda089ce6097fa28e8da4505c6c030000000001000000683ce2664f6da01a786656eb894e9fa63f710603ad3fcb606512f33a3801a5b40200000000ffffffff0220a10700000000002251209c4e1af0245ce5eace1cc1432fe0d0140ca6de1e289aa4e5d34c19d4b7efafdf54959800000000002251204fbb9eda1b53de6589402bd2f5d6a1a9bbf79a9c61e4bfca500c7abe23a25f6ca3000000
{
  "txid": "19a448a1f04cbd8d08d18d790f516e7e613f4903d3349609fe0643588416caef",
  "hash": "19a448a1f04cbd8d08d18d790f516e7e613f4903d3349609fe0643588416caef",
  "version": 2,
  "size": 178,
  "vsize": 178,
  "weight": 712,
  "locktime": 163,
  "vin": [
    {
      "txid": "6c5c50a48d8ea27f09e69c08da5e44da810de8112ea64cce9756fa27c2bdf16b",
      "vout": 3,
      "scriptSig": {
        "asm": "",
        "hex": ""
      },
      "sequence": 1
    },
    {
      "txid": "b4a501383af3126560cb3fad0306713fa69f4e89eb5666781aa06d4f66e23c68",
      "vout": 2,
      "scriptSig": {
        "asm": "",
        "hex": ""
      },
    

### Sign the alice sweeper input (Input 1 - key path spend)

The HTLC input signatures were already created earlier and can be reused. We only need to sign the alice_change input using key path spending.

In [11]:
# SIGHASH for key path spend
sighash_epoch = bytes.fromhex("00")
hash_type = bytes.fromhex("00")  # SIGHASH_DEFAULT (SIGHASH_ALL)

# Transaction data - now includes both inputs
alice_sweeper_value = int(sweeper_initial_fund * 100000000)
alice_sweeper_value = alice_sweeper_value.to_bytes(8, byteorder="little", signed=False)
sha_prevouts_v2 = sha256(commitment_txid + commitment_index + alice_txid_to_spend_bytes + sweeper_index_bytes).digest()
sha_amounts_v2 = sha256(htlc_output_value + alice_sweeper_value).digest()

# scriptPubKeys for both inputs
alice_offered_htlc_spk_with_len = varint_len(alice_offered_htlc_spk) + alice_offered_htlc_spk
alice_sweeper_spk = bytes.fromhex("51") + varint_len(alice_sweeper_pubkey.get_bytes(bip340=True)) + alice_sweeper_pubkey.get_bytes(bip340=True)
alice_sweeper_spk_with_len = varint_len(alice_sweeper_spk) + alice_sweeper_spk
sha_scriptpubkeys_v2 = sha256(alice_offered_htlc_spk_with_len + alice_sweeper_spk_with_len).digest()

sha_sequences_v2 = sha256(sequence_htlc + sequence_change).digest()
sha_outputs_v2 = sha256(outputs_v2).digest()

# Data about this input (key path spend)
spend_type = bytes.fromhex("00")  # key path, no annex

# Index of the input being signed (input 1, not the vout being spent!)
input_index = bytes.fromhex("01000000")

sig_msg_sweeper = (
    sighash_epoch
    + hash_type
    + version
    + locktime
    + sha_prevouts_v2
    + sha_amounts_v2
    + sha_scriptpubkeys_v2
    + sha_sequences_v2
    + sha_outputs_v2
    + spend_type
    + input_index
)

tag_hash = sha256("TapSighash".encode()).digest()
sighash_sweeper = sha256(tag_hash + tag_hash + sig_msg_sweeper).digest()

# Sign with alice_change_privkey
import secrets
aux_sweeper = secrets.token_bytes(32)
alice_sweeper_sig = alice_sweeper_privkey.sign_schnorr(sighash_sweeper, aux_sweeper)

# Verify the signature
alice_sweeper_sig_valid = alice_sweeper_pubkey.verify_schnorr(alice_sweeper_sig, sighash_sweeper)
print("Alice sweeper signature valid?", alice_sweeper_sig_valid)

Alice sweeper signature valid? True


### Build the final signed transaction

Now we construct the witness data for both inputs and build the final signed transaction.

In [12]:
# Witness for Input 1 (alice sweeper) - key path spend
witness_input1 = (
    bytes.fromhex("01")  # 1 stack item
    + varint_len(alice_sweeper_sig)
    + alice_sweeper_sig
)

# Complete witness data
witness_v2 = witness + witness_input1

# The final signed transaction
signed_tx_v2 = (
    version
    + marker
    + flag
    + input_count
    + inputs_v2
    + output_count_v2
    + outputs_v2
    + witness_v2
    + locktime
)

print("signed tx v2:", signed_tx_v2.hex())

# Decode the signed transaction
decoded_signed_v2 = node.decoderawtransaction(signed_tx_v2.hex())
print("\n" + json.dumps(decoded_signed_v2, indent=2, default=str))

signed tx v2: 020000000001026bf1bdc227fa5697ce4ca62e11e80d81da445eda089ce6097fa28e8da4505c6c030000000001000000683ce2664f6da01a786656eb894e9fa63f710603ad3fcb606512f33a3801a5b40200000000ffffffff0220a10700000000002251209c4e1af0245ce5eace1cc1432fe0d0140ca6de1e289aa4e5d34c19d4b7efafdf54959800000000002251204fbb9eda1b53de6589402bd2f5d6a1a9bbf79a9c61e4bfca500c7abe23a25f6c044146c08745896cf49dd44df569af2dcf0db716fa3451c5e78907c63a82048dbf3ab535b8b0001550be40b43a5191ed079d2fe666bf8486aaf6e61a3c477500693783418b3c08268c5da6cdc603c678cee2bf3fafab9c28c71b811f9f98a3771aa035ae6877b146ba7661cec40a49c00e338f2ad68cb140262bb3606d010eb19496811e8344207546c6ab3a8c005ce24e9feb7154fd35b62ac0f9125beb5e692d4d912a95de77ad207bd8383df06640c0efe760bb2bac1a1585bdc664e6f1cb61c41a76a6f2be3484ac41c1c39a6b0cffa569f243cec8252a1e5f93b4b072247ff18773917b72fb6341f6088da58f2aee0bccdd60c6065603a5d2ab655e79971b068228afbccbb02220f54b01402409ceba5222a5278a8f5f0b0438fbd78c014daa6337db5e3737747713d0da2f008151562d29cfae0af3a2e5d0b787

In [13]:
# Test mempool accept
print("\nTest mempool accept:")
try:
    result = node.testmempoolaccept(rawtxs=[signed_tx_v2.hex()])
    print(json.dumps(result, indent=2, default=str))
    
    if result[0]['allowed']:
        print("\n✓ Transaction is valid and meets minimum relay fee!")
        print(f"  Fee: {result[0].get('fees', {}).get('base', 'N/A')} BTC")
    else:
        print(f"\n✗ Transaction rejected: {result[0].get('reject-reason', 'Unknown')}")
except Exception as e:
    print(f"Error: {e}")


Test mempool accept:
[
  {
    "txid": "19a448a1f04cbd8d08d18d790f516e7e613f4903d3349609fe0643588416caef",
    "wtxid": "7e664a91655aeee51f53cfca035436f72cb51eb91866b045314ac5f206f60fe7",
    "allowed": true,
    "vsize": 262,
    "fees": {
      "base": "0.00000300",
      "effective-feerate": "0.00001145",
      "effective-includes": [
        "7e664a91655aeee51f53cfca035436f72cb51eb91866b045314ac5f206f60fe7"
      ]
    }
  }
]

✓ Transaction is valid and meets minimum relay fee!
  Fee: 0.00000300 BTC


## Summary

In this chapter, we successfully created an HTLC-timeout transaction that spends the offered HTLC output from Alice's commitment transaction. Key points:

1. **Purpose**: The HTLC-timeout transaction allows Alice to reclaim funds from an offered HTLC after the timeout period if Bob doesn't provide the payment preimage.

2. **Script Path Spending**: Unlike the commitment transaction which uses key path spending, the HTLC-timeout uses script path spending through the Taproot tree.

3. **Two Signatures Required**: The timeout script (scriptA) requires signatures from both Alice's and Bob's HTLC keys, ensuring both parties agree to the timeout.

4. **Delayed Output**: The output of the HTLC-timeout transaction is similar to the `to_local` output in commitment transactions - it's spendable by Alice after a delay (to_self_delay), or immediately by Bob if he has the revocation key.

5. **Witness Structure**: The witness includes both signatures, the script being executed, and a control block proving the script is part of the Taproot tree.
